# Flask App for SQLIA Detection

From our modeling notebook:
- We have trained a BERT model on SQLIA detection which gives the highest F1-score among all other models
- BERT model F1-score = 0.9977 at a threshold=0.3

Our trained model is present in 'model.pkl' which are going to use for predicting whether our query is SQLIA or not.

Drive link to demo app and model.pkl file: https://drive.google.com/drive/folders/15uctpzZqOFHY2sl0yqv3E3JV4-sWxolv?usp=sharing


In [ ]:
!pip install tensorflow-text

Run these commands to install ngrok. 
- To use ngrok, you have to create an account in https://ngrok.com/. 
- You can find your authentication token in https://dashboard.ngrok.com/get-started/your-authtoken.



```
!pip install pyngrok==4.1.1
!ngrok authtoken <your-auth-token>
```



In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken <your-auth-token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## Creating html templates

We are creating the html templates for our application here. The code below creates the following directory structure:



```
>Templates
...>index.html
...>result.html
```



- We input our SQL query in index.html
- We get the prediction in result.html

In [ ]:
# Creating a new directory Templates
!rm -rf Templates
!mkdir Templates

In [ ]:
# Creating Templates/index.html
with open('Templates/index.html', 'w') as f:
  f.write('''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>SQL Injection Predictor</title>
</head>
<body>

    <h1>SQL Injection Predictor</h1>
    <p>Model that predicts whether a SQL query is an injection or not.</p>
    <form action="predict" method="post">
        <label for="query">SQL Query:</label>
        <input type="text" id="query" name="query"><br><br>
        <input type="submit" value="Submit">
    </form>
</body>
</html>''')
  

# Creating Templates/result.html
with open('Templates/result.html', 'w') as f:
  f.write('''Your query "<strong>{{query}}</strong>" {{prediction}}
<br>
<br>
Execution time: {{extime}} seconds''')

## Creating the Flask app

In [ ]:
# https://stackoverflow.com/questions/66030868/error-rendering-html-template-with-flask-and-google-colab
# https://stackoverflow.com/questions/70342717/colab-unable-to-access-webpage-using-flask-ngrok


# importing libraries
from flask import Flask, request, redirect, url_for
import flask
from flask_ngrok import run_with_ngrok
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
import re
import tensorflow_text as text
import joblib
import time


# Model prediction functions
num_reg = re.compile(r'[0-9]+')
hex_reg = re.compile(r'0x[A-Fa-f0-9]+\b') # regex to match hexadecimal numbers

# function to preprocess the query
def replace_num_hex(query):
    q = num_reg.sub('<num>', query)
    q = hex_reg.sub('<hex>', q)
    return q

# loading the model
model = joblib.load('model.pkl')

# setting optimum threshold
OPTIMUM_THRESHOLD = 0.3

# function to predict if query is SQLI
def model_predict(query):
  """
  Predicts if the query is SQLI or not.
  Inputs:
  query(str): SQL query
  Returns:
  prediction(bool): True if query is SQLI
  """
  try:
    assert type(query) == str
    preprocessed_input = replace_num_hex(query)
    model_input = np.ravel(preprocessed_input)
    model_output = model.predict(model_input)
    prediction = model_output[0][0] > OPTIMUM_THRESHOLD
    return prediction
  except AssertionError:
    print("Enter str input. Current input is of type {}.".format(type(query)))


# Flask App
app = Flask(__name__, template_folder='/content/Templates')
run_with_ngrok(app)


@app.route("/")
def index():
    return flask.render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():

    in_query = request.form['query']
    start_time = time.time()
    is_sqli = model_predict(in_query)
    if is_sqli:
        prediction = "is SQLI"
    else:
        prediction = "is not SQLI"
    execution_time = time.time() - start_time
    return redirect(url_for('result', query=in_query, prediction=prediction, extime=round(execution_time,3)))


@app.route('/result/<query>/<prediction>/<extime>', methods=['GET'])
def result(query, prediction, extime):
    return flask.render_template('result.html', query=query, prediction=prediction, extime=extime)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3ca0-34-125-97-11.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Apr/2022 08:20:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2022 08:20:57] "GET /favicon.ico HTTP/1.1" 404 -


127.0.0.1 - - [10/Apr/2022 08:21:44] "POST /predict HTTP/1.1" 302 -
127.0.0.1 - - [10/Apr/2022 08:22:03] "GET /result/OR%201%3D1/is%20SQLI/0.472 HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2022 08:23:21] "POST /predict HTTP/1.1" 302 -
127.0.0.1 - - [10/Apr/2022 08:23:22] "GET /result/OR%201%3D1/is%20SQLI/0.099 HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2022 08:23:32] "POST /predict HTTP/1.1" 302 -
127.0.0.1 - - [10/Apr/2022 08:23:32] "GET /result/abc123/is%20not%20SQLI/0.092 HTTP/1.1" 200 -
